- **Supported Algorithms**
1. **`invdist` (Inverse Distance Weighting - IDW)**
   - `power`: Determines the weighting of distances. Higher values give more influence to nearby points.
   - `radius1` and `radius2`: Search radii for nearby points in X and Y directions.
   - `nodata`: Value assigned where interpolation is not possible.

2. **`nearest` (Nearest Neighbor)**
   - Assigns values of the nearest point to each grid cell.
3. **`average` (Simple Averaging)**
   - Averages the values of nearby points within the search radius.

4. **`linear` (Linear Interpolation)**
   - Generates a triangulated irregular network (TIN) for linear interpolation.


In [ ]:
from osgeo import gdal
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder created at: {folder_path}")
    else:
        print(f"Folder already exists at: {folder_path}")

def get_raster_file_list(path):
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        # "32628" is just to get here necessary ones
        if file.endswith(".tif") or file.endswith(".tiff"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def get_vector_file_list(path):
    File_list = [] #f for f in os.listdir(path) if os.isfile(mypath,f)
    for file in os.listdir(path):
        # "32628" is just to get here necessary ones
        if file.endswith(".shp"):
            if file not in File_list:
                File_list.append(os.path.join(path,file))
        else:
            pass
    return File_list

def interpolation(input_file, output_path, zfield):
    """
    Creates an interpolation of a vector file using GDAL's Grid utility.

    Parameters:
    input_file (str): The path to the input vector file (e.g., a shapefile) to be interpolated.
    output (str): The path for the output raster file.
    zfield (str): The name of the field in the input vector dataset that contains the values to interpolate.

    Returns:
    output(str): The path for the output raster file.
    """
    # dataSource = ogr.Open(file, 0) # 0 = read mode
    # layer = dataSource.GetLayer()
    output_file = os.path.basename(input_file).replace(".shp",".tif")
    output = os.path.join(output_path, output_file)
    
    # Example :power=3:radius1=2000:radius2=2000"
    nn = gdal.Grid(output, input_file, zfield=zfield, algorithm = "invdist:power=3:radius1=0.07:radius2=0.07:nodata=0",
    width = 100,
    height = 100
    )
    nn = None
    print(f"Finished with: {os.path.basename(output)}")
    return output

def mask_raster(input_file, output_path, cutfile):
    output_file = os.path.basename(input_file).replace(".tif","_clip.tif")
    output = os.path.join(output_path, output_file)

    ds_clipped = gdal.Warp(output, 
        format = 'Gtiff', 
        cutlineDSName=cutfile,
        cropToCutline=True,
        srcDSOrSrcDSTab=input_file,
        creationOptions = ['COMPRESS=DEFLATE','TILED=YES','BLOCKXSIZE=128','BLOCKYSIZE=128']) 

    ds_clipped = None
    return (f"Finished with: {os.path.basename(output)}")


In [ ]:
"""Define the inputs"""
input_path = r""
interpolation_path = input_path + "\inpterpolation_path"
masking_path = input_path + "\masking_path"

cutfile = r""

In [ ]:
create_folder_if_not_exists(interpolation_path)
create_folder_if_not_exists(masking_path)

vector_file_list = get_vector_file_list(input_path)
for file in vector_file_list:
    output_interpolation = interpolation(file, interpolation_path, "value_name")
    mask_raster(output_interpolation, masking_path, cutfile)
